# Shifting science frames

Written by Evgenii N.

The following code shifts the science frames so that stars appear at same x-y positions in all images. The input data is in `030_science_frames/data/reduced` directory and the shifted images are saved to `040_shift/data/shifted`.

## Prerequisite code

In [1]:
# Import libraries that we will use later in this notebook
import os
import shutil
import ccdproc
import numpy as np
from astropy.visualization import ZScaleInterval, MinMaxInterval, ImageNormalize
from astropy import units as u
from matplotlib.colors import LogNorm
from ccdproc import CCDData
import matplotlib.pyplot as plt
from photutils.aperture import CircularAperture, aperture_photometry
from photutils.centroids import centroid_2dg
from scipy.ndimage import shift

# Make images non-blurry on high pixel density screens
%config InlineBackend.figure_format = 'retina'

# Title size
plt.rcParams['axes.titlesize'] = 16

# Axes label size
plt.rcParams['axes.labelsize'] = 13


def show_image(image, title):
    """
    Display an image.
    
    Parameters
    ---------
    
    image: astropy.nddata.ccddata.CCDData
        A fits image to show.
        
    title: str
        Plot title.
    """
    fig, ax = plt.subplots(figsize=(12, 8))  # Change image size
    plt.rcParams.update({'font.size': 10})  # Change font size
    
    # Scale the image similar to 'zscale' mode in DS9.
    # This makes easier to spot things in the image.
    interval=ZScaleInterval()
    vmin, vmax = interval.get_limits(image)
    norm = ImageNormalize(vmin=vmin, vmax=vmax)
    
    plt.imshow(image, cmap='gray', norm=norm)  # Set color map and pixel scaling
    plt.xlabel('x [pixel]')  # Set axis labels
    plt.ylabel('y [pixel]')
    plt.title(title, y=-0.2)  # Set image title
    plt.colorbar()  # Show color bar
    

def print_image_stats(image, title):
    """
    Print first pixel value, average and standard deviation for an image.
    
    Parameters
    ---------
    
    image: astropy.nddata.ccddata.CCDData
        A fits image to show.
        
    title: str
        Image name.
    """
    
    data = np.asarray(image)# Get numpy array for image data
    label_len = 10  # Length of the text label
    first_pixel = data[0, 0]  # First pixel
    average = np.mean(data)  # Average
    standard_deviation = np.std(data)   # Standard deviation


    # Print values
    # -------

    print(
        f'\n{title}',
        f"\n{'-' * len(title)}",
        f"\n{'Pixel:':<10}{first_pixel:>10.2f} ADU",
        f"\n{'Avg:':<10}{average:>10.2f} ADU",
        f"\n{'Std:':<10}{standard_deviation:>10.2f} ADU\n"
    )
    

def save_image(image, file_path):
    """
    Save image to disk. Overwrites the file if it already exist.
    
    Parameters
    ---------
    image: astropy.nddata.ccddata.CCDData
        Image to be saved
        
    file_path: str
        Path where the image is saved
    """
    
    # Delete the file if it already exists
    
    try:
        os.remove(file_path)
    except OSError:
        pass
    
    # Create directory
    # ------

    dirname = os.path.dirname(file_path)
    
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    image.write(file_path)

## Program code

In [15]:
def load_images_from_dir(dir_path, include):
    """
    Loads fits images from the directory.
    
    Parameters
    ----------
    
    dir_path: str
        Path where the images are loaded from.
        
    include: str
        The pattern to filter the file names, e.g. '*.fit'.
        
    Returns
    --------
    (images, image_paths)
        images: list of astropy.nddata.ccddata.CCDData
            Loaded images.
        image_paths: list of str
            Image names.
    """

    images = ccdproc.ImageFileCollection(dir_path, glob_include=include)

    # Make sure we are only reading science images
    images = images.files_filtered(PICTTYPE=1) 
    
    # Read the images
    return [
                CCDData.read(os.path.join(dir_path, image))
                for image in images
           ], images


def calculate_star_positions(images, box_center, box_size):
    """
    Calculate position of the star in `images` by searching
    within a box of size `box_size` centered at `star_position`.
    
    Parameters
    ----------
        
    images: list of astropy.nddata.ccddata.CCDData
        List of images.
        
    box_center: (x, y)
        Coordinates of the center of the search box.
        
    box_size: float
        The size of the search box.
        
    Returns
    --------
    list of (x, y)
        x, y: loat
            Positions of the star in the images.
    """
        
    half_box = int(box_size / 2)  # Half the size of the search box
    star_positions = []

    # Loop over the images
    for image, name in zip(images, image_names): 
        # Make copy of the image
        image = image.copy()

        # Subtract the background
        image = image - np.ma.median(image)
        
        # Get the image region where we will search for a star
        # Note: Y coordinates come first
        search_box = image[box_center[1] - half_box: box_center[1] + half_box,
                           box_center[0] - half_box: box_center[0] + half_box]

        # Estimate the pixel coordinates of the star, within the search box
        x_box, y_box = centroid_2dg(search_box)
        
        # Calculate x and y cooridinates of the star realtive to the image
        x = box_center[0] - half_box + x_box
        y = box_center[1] - half_box + y_box
        star_positions.append((x, y))
        
    return star_positions


def shift_images(images, shifts): 
    """
    Shift the images.
    
    Parameters
    ----------
    
    images: list of astropy.nddata.ccddata.CCDData
        List of images to be shifted.
        
    shifts: list of (dx, dy)
        Shift amount for each image.
        
        
    Returns
    -------
    list of astropy.nddata.ccddata.CCDData
        Shifted images.
    """

    shifted_images = []

    for image, xy_shift in zip(images, shifts): 
        # Get the star offset for this image relative to the first image
        # Note, x/y coordinates are flipped.
        yx_shift = (xy_shift[1], xy_shift[0])

        # Shift the image data
        shifted = shift(image, yx_shift, order=0, mode='constant', cval=-100)
        shifted_images.append(shifted)
        
    return shifted_images

In [16]:
# Set Bias and Dark image paths
# -------

source_dir = '../030_science_frames/data/reduced'
data_dir = './data/'

# Shift images
# -------

nights = ['march_09_2018', 'march_29_2018', 'april_30_2018']
nights = ['april_30_2018']

# Shift images for all nights
for night in nights:
    dir_path = os.path.join(source_dir, night)
    images, image_names = load_images_from_dir(dir_path=dir_path, include='*.fit')
    
    box_center = (554, 405)
    
    # Detect a star within a box in each image and calculate its position
    positions = calculate_star_positions(images=images,
                                         box_center=box_center,
                                         box_size=50)
    
    # Calculate the shifts relative to the first image
    positions = np.array(positions)
    shifts = positions[0] - positions

    # Shift the images
    shifted_images = shift_images(images=images, shifts=shifts)
    
    # Calculate the star position in the shifted images
    shifted_positions = calculate_star_positions(images=shifted_images,
                                                 box_center=box_center,
                                                 box_size=50)
    
    for image_name, position, shift_amount, shifted_position \
        in zip(image_names, positions, shifts, shifted_positions):

        print((
                f'{image_name} '
                f'({position[0]:.0f}, {position[1]:.0f}) '
                f'({shift_amount[0]:.0f}, {shift_amount[1]:.0f}) '
                f'({shifted_position[0]:.0f}, {shifted_position[1]:.0f}) '
              ))
        
        
print("------------------")
print("We are done")

NGC_3201_B_30.000secs_00001305.fit (552, 404) (0, 0) (552, 404) 
NGC_3201_B_30.000secs_00001306.fit (555, 403) (-3, 1) (552, 404) 
NGC_3201_B_5.000secs_00001292.fit (563, 410) (-11, -7) (586, 408) 
NGC_3201_B_5.000secs_00001293.fit (563, 410) (-11, -6) (552, 404) 
NGC_3201_B_5.000secs_00001294.fit (569, 415) (-17, -11) (591, 419) 
NGC_3201_I_30.000secs_00001313.fit (552, 400) (-0, 4) (552, 404) 
NGC_3201_I_30.000secs_00001315.fit (561, 399) (-9, 4) (552, 403) 
NGC_3201_I_5.000secs_00001301.fit (557, 408) (-5, -4) (552, 404) 
NGC_3201_I_5.000secs_00001302.fit (557, 408) (-5, -4) (552, 404) 
NGC_3201_I_5.000secs_00001303.fit (557, 406) (-5, -2) (552, 404) 
NGC_3201_R_30.000secs_00001310.fit (557, 402) (-5, 1) (552, 403) 
NGC_3201_R_30.000secs_00001312.fit (550, 401) (2, 3) (552, 404) 
NGC_3201_R_5.000secs_00001298.fit (558, 409) (-7, -5) (551, 404) 
NGC_3201_R_5.000secs_00001299.fit (557, 409) (-6, -5) (551, 404) 
NGC_3201_R_5.000secs_00001300.fit (558, 408) (-6, -5) (552, 403) 
NGC_3201